In [11]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [12]:
from crewai import Agent, Task, Crew

In [13]:
import os
from langchain.llms import Ollama
from dotenv import load_dotenv
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = Ollama(
    model = "llama2",
    base_url = "http://localhost:11434")
    

In [14]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Provide the best customer service to our clients",
    backstory="You work at crewAI (https://crewai.com), and provide support  "
              "to {customer}. You need to provide full complete answers, and make no assumptions.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [15]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Review the quality of the support provided by the Senior Support Representative",
    backstory="You work at crewAI (https://crewai.com), and review the quality of the support provided by the Senior Support Representative. "
              "You need to provide feedback on the quality of the answers, and make no assumptions.",
    verbose=True,
    llm=llm
)

In [16]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [17]:
inquiry_resolution = Task(
    description="Resolve the {inquiry} of {person} from {customer}.",
    expected_output="A detailed, informative response that address the {inquiry} of {customer}. "
                    "The response should be clear, concise, and accurate. "
                    "The response should be reviewed by the Support Quality Assurance Specialist. ",
    agent=support_agent
)

In [18]:
quality_assurance_review = Task(
    description="Review the response to {customer}'s inquiry of the support provided by the Senior Support Representative."
                "Ensure that the response is clear, concise, and accurate."
                "Provide feedback to the Senior Support Representative.",
    expected_output="A detailed feedback on the quality of the response provided by the Senior Support Representative.",
    agent=support_quality_assurance_agent
)

In [19]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=True,
    embedder={
        "provider": "huggingface",
        "config": {
                "model": "mixedbread-ai/mxbai-embed-large-v1", # https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
        }
    }
)

2024-08-01 18:22:31,976 - 140704365273024 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
llm.base_url = "http://localhost:11434"
inputs  = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "How to create a Crew and kick it off?"
}  # Define the inputs variable with appropriate values
result = crew.kickoff(inputs=inputs)

 [2024-08-01 18:22:42][DEBUG]: == Working Agent: Senior Support Representative
 [2024-08-01 18:22:42][INFO]: == Starting Task: Resolve the How to create a Crew and kick it off? of Andrew Ng from DeepLearningAI.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: To create a crew and kick it off in DeepLearningAI, follow these steps:

Step 1: Define the Crew's Purpose and Objectives
The first step is to clearly define the purpose and objectives of the crew. What problem are you trying to solve? What are the goals and key performance indicators (KPIs) that you want to achieve? Be specific and measurable in your definitions.

Step 2: Identify the Right People for the Crew
Once you have a clear understanding of the purpose and objectives, identify the right people to join the crew. These should be individuals with diverse skill sets and expertise who can contribute to achieving the crew's goals. Consider using tools like LinkedIn or other profes

KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
Markdown(result)